# DenseNet

In [ ]:
!pip install ipywidgets --upgrade

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
from torchvision.models import densenet121 

In [ ]:
class CustomDenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super(CustomDenseBlock, self).__init__()
        self.layers = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, growth_rate, kernel_size=3, stride=1, padding=1, bias=False)
        )

    def forward(self, x):
        out = self.layers(x)
        out = torch.cat([x, out], 1) 
        return out

# DenseNet architecture
class CustomDenseNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomDenseNet, self).__init__()
        self.conv0 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn0 = nn.BatchNorm2d(64)  
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.dense1 = self._make_dense_block(64, 64, 6) 
        self.transition1 = self._make_transition_layer(64 + 6 * 64, 128) 
        self.dense2 = self._make_dense_block(128, 64, 12)
        self.transition2 = self._make_transition_layer(128 + 12 * 64, 256) 
        self.dense3 = self._make_dense_block(256, 32, 24)
        self.transition3 = self._make_transition_layer(256 + 24 * 32, 512)  

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_dense_block(self, in_channels, growth_rate, num_layers):
        layers = []
        for _ in range(num_layers):
            layers.append(CustomDenseBlock(in_channels, growth_rate))
            in_channels += growth_rate
        return nn.Sequential(*layers)

    def _make_transition_layer(self, in_channels, out_channels):
        return nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        x = self.conv0(x)
        x = self.bn0(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.dense1(x)
        x = self.transition1(x)
        x = self.dense2(x)
        x = self.transition2(x)
        x = self.dense3(x)
        x = self.transition3(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [ ]:
transform = transforms.Compose([
    transforms.Grayscale(1),
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.RandomCrop(size=(60, 60)),  
    transforms.GaussianBlur(kernel_size=3),  
    transforms.RandomResizedCrop(size=(64, 64), scale=(0.8, 1.0)),  
    transforms.ToTensor()
])

train_dataset = ImageFolder(root='/kaggle/input/fer2013/train', transform=transform)
test_dataset = ImageFolder(root='/kaggle/input/fer2013/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 7
model = CustomDenseNet(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

def train_model(model, criterion, optimizer, train_loader, test_loader, device, epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
        
        epoch_loss = running_loss / len(train_loader.dataset)
        train_accuracy = correct_train / total_train * 100
        
        train_losses.append(epoch_loss)
        train_accuracies.append(train_accuracy)
        
        print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {epoch_loss:.4f} - Train Accuracy: {train_accuracy:.2f}%")

        test_metrics = test_model(model, criterion, test_loader, device)
        test_losses.append(test_metrics['loss'])
        test_accuracies.append(test_metrics['accuracy'])
        
   # Plotting training and test losses side by side
    plt.figure(figsize=(15, 5))

    # Subplot for training and test losses
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(test_losses, label='Test Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Test Losses')
    plt.legend()

    # Subplot for training and test accuracies
    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Train Accuracy')
    plt.plot(test_accuracies, label='Test Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training and Test Accuracies')
    plt.legend()


In [ ]:
def test_model(model, criterion, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)

    test_loss = running_loss / len(test_loader.dataset)
    test_accuracy = correct / total * 100

    print(f"Test Loss: {test_loss:.4f} - Test Accuracy: {test_accuracy:.2f}%")
    
    return {'loss': test_loss, 'accuracy': test_accuracy}


In [ ]:
train_model(model, criterion, optimizer, train_loader, test_loader, device, epochs=50)

In [ ]:
torch.save(model.state_dict(), 'DNetADAM.pth')

## SGD

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 7
model = CustomDenseNet(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-5)

In [ ]:
train_model(model, criterion, optimizer, train_loader, test_loader, device, epochs=10)

In [ ]:
torch.save(model.state_dict(), 'DNetSGD.pth')